# Similaridade entre os nomes dos artigos


Primeiramente, iremos encontrar todos os titulos dos artigos da string de busca que estamos interessados, através do programa [exampleProg2.py](exampleProg2.py). Para isto, basta executar o programa e preencher o campo input() com a string desejada.

**OBSERVAÇÃO: Caso a busca retorne mais que 5000 resultados, aparentemente a pyscopus não suporta tal busca e o programa resulta em um erro.**

Após preenchido o input() com a string desejada, acesse o arquivo [Resultado.csv](resultado.csv) onde estarão alocados, um por linha, todos os títulos retornados pela busca. Segue o código do arquivo exampleProg2.py.

In [ ]:
from pyscopus import Scopus
import pandas as pd
import re

#Quantos resultados quero que retorne?
resultados = 5000

#Chave de acesso Scopus
key = '56c667e47c588caa7949591daf39d8a0'

scopus = Scopus(key)

#Recebendo string de busca e formatando-a adequadamente
string = raw_input("Digite a string desejada:\n")
#string = re.sub(r'(")', r'\"', string)

#Executando a busca na SCOPUS com o pyscopus
search_df = scopus.search(string, count = resultados)

#print(search_df)

#Editando os parametros do dataframe retornado
# https://pandas.pydata.org/pandas-docs/stable/options.html
pd.options.display.max_rows = 99999
pd.options.display.max_colwidth = 250

# https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_csv.html

#print(search_df[['title']])
search_df[['title']].to_csv("Resultado.csv", index_label=False, encoding = 'utf-8', index=False, header=True, sep='\t')

Após obter os nomes dos artigos no arquivo [Resultado.csv](resultado.csv), executa-se o arquivo [SimilaridadeLeo.py](SimilaridadeLeo.py), que recebe como entrada os arquivos [QGS.csv](QGS.csv) e  [Resultado.csv](Resultado.csv) e gera um arquivo de saída denominado [Saida.csv](Saida.csv) que contém os artigos do QGS ordenados, e abaixo destes os arquivos mais próximos deste obtidos do resultado da busca.

Com isso, conseguimos realmente avaliar se um arquivo do QGS foi encontrado pela busca ou não, ficando mais prático a conferência de quantos arquivos do QGS se enquadram na busca.

In [ ]:
# encoding: utf-8

import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

QGS = pd.read_csv('/home/fuchs/Documentos/MESTRADO/Masters/QGS.csv', sep='\t')
Resultado = pd.read_csv('/home/fuchs/Documentos/MESTRADO/Masters/Resultado.csv', sep='\t')

Saida = open('/home/fuchs/Documentos/MESTRADO/Masters/Saida.csv', 'w')


#with open('/home/fuchs/Documentos/MESTRADO/Masters/QGS.csv') as csvfile:
#    QGS = list(csv.reader(csvfile))

#with open('/home/fuchs/Documentos/MESTRADO/Masters/Resultado.csv') as csvfile:
#    Resultado = list(csv.reader(csvfile))

lenQGS = sum(1 for line in open('/home/fuchs/Documentos/MESTRADO/Masters/QGS.csv')) - 1
lenResultado = sum(1 for line in open('/home/fuchs/Documentos/MESTRADO/Masters/Resultado.csv')) - 1
#QGS = [val for sublist in QGS for val in sublist]

listaQGS = []
listaResultado = []

for i in range (0, lenQGS):
    listaQGS.append(QGS.iloc[i, 0])

#print("Lista QGS:", listaQGS)
#print("Tamanho Lista QGS:", len(listaQGS))

for i in range (0, lenResultado):
    listaResultado.append(Resultado.iloc[i, 0])

#print("Lista Resultado:", listaResultado)
#print("Tamanho Lista Resultado:", len(listaResultado))

train_set = [listaQGS, listaResultado]
train_set = [val for sublist in train_set for val in sublist]

#print("Lista train_set:", train_set)
#print("Elementos train_set", len(train_set))

tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix_train = tfidf_vectorizer.fit_transform(train_set)

matSimilaridade = cosine_similarity(tfidf_matrix_train[0:lenQGS], tfidf_matrix_train[lenQGS:lenQGS+lenResultado])
lin, col = matSimilaridade.shape

#print ("Tamanho da matriz de similaridade:", matSimilaridade.shape)

#IMPRIMIR MATRIZ
#for i in range (0, lin):
#    linha = matSimilaridade[i]
#    print(linha)

for i in range (0, lin):
    linha = matSimilaridade[i]
    currentNearest = np.argsort(linha)[-4:] #Pega os x - 1 maiores elementos
    linhaSaida = 'Artigo do QGS:  ' + listaQGS[i] + '\t' + '\n'
    for i in range(1, len(currentNearest)):
        book = currentNearest[-i]
        linhaSaida = linhaSaida + '\t\t\t\t' + listaResultado[book].strip() + '\t' '\n'
    linhaSaida = linhaSaida + "\n"
    Saida.write(linhaSaida)
    Saida.flush()

#print cosine_similarity(tfidf_matrix_train[0], tfidf_matrix_train)